# Run local LLMs with Ollama and Langchain

- [Install Ollama locally](https://github.com/ollama/ollama)
- Test

`ollama pull llama3`

`ollama run llama3`

- Available models for native download - https://ollama.com/library
- I also used [LM Studio](https://lmstudio.ai/) to search for and download models from HuggingFace , including community optimized quantized models
- For Ollama download a gguf from huggingface, create a [Modelfile](https://github.com/ollama/ollama/blob/main/docs/modelfile.md), then e.g. 

`ollama create mixtral -f Modelfile`

- You can [quantize and fine-tune your own](https://www.answer.ai/posts/2024-03-06-fsdp-qlora.html)
- This is run on M3 Macbook Air with 24GB memory. The 9b-parameter Gemma2 model performs well on these tests. The 27b-parameter model is probably pretty amazing, but the biggest one I can run on this MacBook is 3-bit K-quantization (S) - says not recommended - https://huggingface.co/bartowski/gemma-2-27b-it-GGUF
- They run fast enough on M3 as long as you stick to models LM Studio says 'Full GPU Offload Possible'


In [72]:
import os
import dotenv
import warnings
from datetime import datetime, timedelta
import sys
import yaml

import multiprocessing
from concurrent.futures import ThreadPoolExecutor, as_completed

import pandas as pd

import IPython
from IPython.display import HTML, Image, Markdown, display

from bs4 import BeautifulSoup

from langchain_openai import ChatOpenAI
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.prompts import PromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain.memory import ChatMessageHistory
from langchain.tools import StructuredTool
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser

import openbb
from openbb import obb
from openbb_core.app.model.obbject import OBBject

import openai
from openai import OpenAI
import tiktoken

import langchain
from langchain_core.tools import StructuredTool
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

# langchain is still on pydantic v1
import pydantic.v1
from pydantic.v1 import BaseModel, Field # <-- Uses v1 namespace

import wikipedia

# free API for edgar filing
from sec_downloader import Downloader
import sec_parser as sp

try:
    module_name = 'webscrape'
    del sys.modules[module_name]
except:
    pass

import webscrape

dotenv.load_dotenv()

# turn off excessive warnings
warnings.filterwarnings('ignore')


In [2]:
page = 'Eli Lilly and Company'

def fn_get_wiki_page(page):
    """Given the name of existing wikipedia page, return the content of the page"""
    return wikipedia.page(title=page, auto_suggest=False).content

page_content = fn_get_wiki_page(page)
page_content[:1000]
print(len(page_content))


56222


In [3]:
prompts = [
"Translate ‘You are amazing and I love you’ into French.",
"Define ‘photosynthesis’.",
"Copy edit this sentence for grammar, spelling, and punctuation: ‘She decide too bake cookies for her friends but then she realized she didn’t have enough suger.’",
"Write a humorous limerick about hedge funds where the first letter of each verse spells 'hedge'.",
f"""You will act as a securities analyst with deep knowledge of financial markets,
securities analysis, and portfolio management. You will maintain a professional yet engaging tone,
in the style of a senior investment bank research analyst. You will concisely provide facts
relevant to investors and avoid opinions unless specifically requested.

You will focus on Eli Lilly (LLY). You will summarize the provided content below from
Wikipedia for Eli Lilly and summarize all topics of interest to investors on this page in 3000
words or less: {page_content}""",
]


In [4]:
models = [
#     "Yi_lm",
#     "gemma_lm",
#     "yi",
#     "aya",
    "phi3",
    "qwen2",
    "gemma2",
#     "llama3",
#     "mistral",
#     "mixtral",
]


In [5]:
local_llm = ChatOllama(model=models[0], temperature=0)

messages = [
    SystemMessage(content="You are a friendly assistant."),
    HumanMessage(content=prompts[0]),
]

response = local_llm.invoke(messages)

print(response.content, "\n")


 Vous êtes incroyable et j'aime vous. 



In [6]:
system_template = "You are a helpful assistant."

all_responses = ""

for m in models:
    
    print(f"# {m}\n")
    response = f"# {m}\n"
    local_llm = ChatOllama(model=m, temperature=0)
    
    for p in prompts:
        print(p[:200])

        prompt_template = ChatPromptTemplate.from_messages(
            [("system", system_template),
             ("user", p)]
        )

        parser = StrOutputParser()

        chain = prompt_template | local_llm | parser

        # stream tokens as they are generated
        for r in chain.stream({}):
            print(r, end="")
            response += r

        print("\n\n---\n\n")
        response += "\n\n---\n\n"
        
        
    all_responses += response
    all_responses += "\n\n---\n\n"
        
        
# display(Markdown(all_responses))

# response


# phi3

Translate ‘You are amazing and I love you’ into French.
 Vous êtes incroyable et j'aime vous.

---


Define ‘photosynthesis’.
 Photosynthesis is the process by which green plants, algae, and some bacteria convert light energy, usually from the sun, into chemical energy in the form of glucose or other sugars. This complex biochemical reaction occurs primarily within specialized cell structures called chloroplasts found in plant cells.

During photosynthesis, plants use carbon dioxide (CO2) and water (H2O), along with sunlight energy, to produce glucose (C6H12O6) and oxygen gas (O2). The overall chemical equation for this process can be represented as:

6 CO2 + 6 H2O + light energy → C6H12O6 + 6 O2

Photosynthesis consists of two main stages: the light-dependent reactions and the light-independent reactions (also known as the Calvin cycle). In the first stage, sunlight is absorbed by chlorophyll pigments in the thylakoid membranes within the chloroplasts. This energy is used to s

Photosynthesis is the amazing process used by plants, algae, and some bacteria to convert light energy from the sun into chemical energy in the form of sugars. 

Here's a simple breakdown:

* **Sunlight:** Plants capture light energy using a green pigment called chlorophyll.
* **Water:** Plants absorb water from the soil through their roots.
* **Carbon Dioxide:** Plants take in carbon dioxide from the air through tiny pores on their leaves called stomata.

Using the captured light energy, plants combine water and carbon dioxide to create glucose (sugar), which is their food source.  As a byproduct, they release oxygen into the atmosphere.

**In essence, photosynthesis is how plants make their own food and provide the oxygen we breathe!** 


Let me know if you'd like to know more about the specific steps involved in photosynthesis or its importance to life on Earth.

---


Copy edit this sentence for grammar, spelling, and punctuation: ‘She decide too bake cookies for her friends but th

In [7]:
display(Markdown(all_responses))


# phi3
 Vous êtes incroyable et j'aime vous.

---

 Photosynthesis is the process by which green plants, algae, and some bacteria convert light energy, usually from the sun, into chemical energy in the form of glucose or other sugars. This complex biochemical reaction occurs primarily within specialized cell structures called chloroplasts found in plant cells.

During photosynthesis, plants use carbon dioxide (CO2) and water (H2O), along with sunlight energy, to produce glucose (C6H12O6) and oxygen gas (O2). The overall chemical equation for this process can be represented as:

6 CO2 + 6 H2O + light energy → C6H12O6 + 6 O2

Photosynthesis consists of two main stages: the light-dependent reactions and the light-independent reactions (also known as the Calvin cycle). In the first stage, sunlight is absorbed by chlorophyll pigments in the thylakoid membranes within the chloroplasts. This energy is used to split water molecules into hydrogen ions and oxygen gas, while also generating ATP (adenosine triphosphate) and NADPH (nicotinamide adenine dinucleotide phosphate), which are essential for the second stage of photosynthesis.

In the light-independent reactions or Calvin cycle, the energy stored in ATP and NADPH is used to convert carbon dioxide into glucose through a series of enzyme-catalyzed chemical reactions. The produced glucose can then be utilized by plants for growth, reproduction, and other metabolic processes or stored as starch for later use.

Photosynthesis is essential to life on Earth, as it provides the primary source of energy for most living organisms through food chains and contributes significantly to oxygen production in our atmosphere.

---

 She decided to bake cookies for her friends, but then she realized she didn't have enough sugar.

(Note: "decide" was corrected to its base form "decided," and "too" was changed to "to.")

---

 Hedge funds, oh how they grow,
Eager investors in tow.
Dollars dance and sway,
Gains come out on display,
Everyone's wealth just starts to flow!

---

 Eli Lilly and Company, founded in 1876 by Colonel John E. Lilly, is a global biopharmaceecial company headquartered in Indianapolis, Indiana. It has grown to become one of the largest pharmaceutical companies worldwide, with its products spanning across various therapeutic areas including oncology, diabetes care, and neuroscience.

The company's history is marked by significant milestones such as the development of insulin in 1923, which revolutionized diabetes treatment. Eli Lilly also faced legal challenges over marketing practices, including an illegal promotion of Evista for conditions not approved by the FDA and a controversy surrounding rising insulin prices.

Eli Lilly's commitment to innovation is evident in its extensive research and development efforts, with numerous patents granted annually. The company has also been involved in various philanthropic activities, including supporting education through scholarships and grants.

Despite facing criticism for high drug prices, Eli Lilly continues to invest heavily in R&D, aiming to address unmet medical needs and improve patient outcomes. Its recent initiatives include reducing the cost of insulin and offering discounts on certain medications, reflecting a response to public concerns over healthcare affordability.

Eli Lilly's impact extends beyond its products; it has contributed significantly to Indiana's economy and cultural heritage, with historical landmarks like Building 314 in Indianapolis serving as reminders of the company's legacy.

---



---

# qwen2
"Vous êtes incroyable et je vous aime"

---

Photosynthesis is the process used by plants, algae, and certain bacteria to convert light energy into chemical energy that can be used to fuel their growth. This process occurs in the chloroplasts of plant cells, which contain a green pigment called chlorophyll.

During photosynthesis, these organisms absorb carbon dioxide (CO2) from the air and water (H2O) from the soil using sunlight as an energy source. The light energy is captured by chlorophyll and used to convert CO2 and H2O into glucose (C6H12O6), a simple sugar that serves as food for the plant, and oxygen (O2), which is released into the air.

The overall chemical equation for photosynthesis can be represented as:

\[ 6CO_2 + 6H_2O + light \rightarrow C_6H_{12}O_6 + 6O_2 \]

This process not only provides energy for plants but also plays a crucial role in the global carbon cycle and is responsible for producing most of the oxygen found in Earth's atmosphere.

---

"She decided to bake cookies for her friends, but then she realized she didn't have enough sugar."

---

There once was a fund quite bold,
Whose name started with "H" and gold,
In markets it roared,
With strategies galore,
But always kept folks on the hold.  

Its managers, so sly,
Would bet on what's up or low,
With leverage they'd play,
And portfolios they'd sway,
Yet, risks were not to be told.

In tales of its trade,
Some saw wealth in their stride,
While others would dread,
The volatility spread,
For "hedge" was a code for surprise.  

So here ends our rhyme,
Of funds that might climb or decline,
Remember the rule:
When investing, beware the fool,
And hedge your bets with care and line.

---

Eli Lilly and Company is a global pharmaceutical company founded in Indianapolis, Indiana by Colonel Eli Lilly in 1876. The company's history includes significant contributions to the field of medicine through research and development of various drugs.

Key points about Eli Lilly:

- **Innovation**: Eli Lilly has been at the forefront of innovation with numerous breakthroughs including insulin production for diabetes management, antipsychotic medication Zyprexa, and cancer treatments like Evista. 

- **Legal Issues**: The company faced significant legal challenges over marketing practices, particularly concerning its drug Zyprexa which led to large settlements due to allegations of illegal promotion.

- **Patent Disputes**: Eli Lilly has also been involved in patent disputes including a lawsuit against Canada for alleged violations under the North American Free Trade Agreement regarding patents on drugs Strattera and Zyprexa.

- **Insulin Pricing Controversy**: In recent years, Eli Lilly faced criticism over insulin pricing, with accusations of price gouging leading to investigations by US lawmakers. 

- **Corporate Responsibility**: Despite these challenges, Eli Lilly has been recognized for its contributions to healthcare and has implemented measures to ensure integrity in business practices.

Eli Lilly continues to operate globally, contributing significantly to the pharmaceutical industry through research and development while facing ongoing scrutiny over its operations and practices.

---



---

# gemma2
"Tu es incroyable et je t'aime." 


Let me know if you'd like me to translate anything else! 😊  


---

Photosynthesis is the amazing process used by plants, algae, and some bacteria to convert light energy from the sun into chemical energy in the form of sugars. 

Here's a simple breakdown:

* **Sunlight:** Plants capture light energy using a green pigment called chlorophyll.
* **Water:** Plants absorb water from the soil through their roots.
* **Carbon Dioxide:** Plants take in carbon dioxide from the air through tiny pores on their leaves called stomata.

Using the captured light energy, plants combine water and carbon dioxide to create glucose (sugar), which is their food source.  As a byproduct, they release oxygen into the atmosphere.

**In essence, photosynthesis is how plants make their own food and provide the oxygen we breathe!** 


Let me know if you'd like to know more about the specific steps involved in photosynthesis or its importance to life on Earth.

---

She decided to bake cookies for her friends, but then she realized she didn't have enough sugar. 


---

**H**igh-rolling traders, sharp and keen,
**E**ager to make fortunes unseen.
**D**ealing in stocks and more,
**G**etting rich, that's for sure,
**E**ven if the market's a mean machine. 


Let me know if you'd like to try another limerick! 


---

This text provides a comprehensive overview of Eli Lilly and Company, covering its history, community service, controversies, and more. 

Here's a breakdown of the key points:

**History:**

* Founded in 1876 by Colonel Eli Lilly in Indianapolis, Indiana.
* Initially focused on producing morphine and other pharmaceutical products.
* Expanded its product line to include insulin, Prozac, Zyprexa, and other medications.
* Acquired by its current parent company in 1987.

**Community Service:**

* Long history of philanthropy and support for local organizations.
* Provided aid to disaster victims, including the San Francisco earthquake and the Johnstown Flood.
* Established Lilly Field Hospital 32 during World War I.
* Manufactured over 200 products for military use during World War II.

**Foundations:**

* **Lilly Endowment:** Founded in 1937 by Josiah K. Lilly Sr. and his sons, Eli and Joe.
* **Eli Lilly and Company Foundation:** Established in 1968, funded through corporate profits.

**Controversies:**

* **BGH:** Purchase of bovine growth hormone (BGH) from Monsanto in 2008, facing ethical and health concerns.
* **340B:** Legal battle with the Department of Health and Human Services over drug pricing for hospitals.
* **Prozac:** Lawsuits and public outcry following a 1989 mass shooting linked to the drug.
* **Zyprexa:** Multiple lawsuits and criminal charges for illegally marketing the drug for unapproved uses.
* **Discrimination:** Accused of sex discrimination and age discrimination in hiring practices.
* **Canada Patent Lawsuit:** Sued Canada for violating its obligations to foreign investors.
* **Evista:** Guilty plea and fine for illegally promoting the drug for unapproved uses.
* **Insulin Pricing:** Facing criticism for high insulin prices and slow response to calls for affordability.

**Overall:**

The text paints a complex picture of Eli Lilly and Company, highlighting its contributions to medicine and society alongside its involvement in controversies and legal battles. It demonstrates the challenges faced by pharmaceutical companies in balancing innovation, profitability, and ethical considerations.




---



---



In [43]:
import sqlite3

SQLITE_DB = "/Users/drucev/projects/AInewsbot/articles.db"

datestr = '2024-06-30'

conn = sqlite3.connect(SQLITE_DB)

c = conn.cursor()
query = f"select * from news_articles where article_date > '{datestr}' and isAI=1 order by article_date desc"
df = pd.read_sql_query(query, conn)
df


,id,src,title,url,isAI,article_date
0,117263,Bloomberg Tech,China AI Startups Head to Singapore in Bid for...,https://www.bloomberg.com/news/articles/2024-0...,1,2024-07-01
1,117267,Bloomberg Tech,Turkey Turns to AI to Crack Down on Rampant Ta...,https://www.bloomberg.com/news/articles/2024-0...,1,2024-07-01
2,117269,Business Insider,What happens to AI progress if China invades T...,https://www.businessinsider.com/ai-taiwan-chin...,1,2024-07-01
3,117282,FT Tech,Working It’s guide to AI at work,https://www.ft.com/content/85b528a8-282d-4479-...,1,2024-07-01
4,117292,Feedly AI,Palantir's Peter Thiel Says It's 'Very Strange...,https://www.benzinga.com/news/24/07/39569675/p...,1,2024-07-01
...,...,...,...,...,...,...
139,117652,Reddit,"Not bad, still not really usable but not bad a...",https://www.reddit.com/r/ChatGPT/comments/1dsi...,1,2024-07-01
140,117653,Reddit,You can generate GIFs with ChatGPT directly!,https://www.reddit.com/r/ChatGPT/comments/1dse...,1,2024-07-01
141,117654,Reddit,I Made A List Of 100+ Words & Phrases That Cha...,https://www.reddit.com/r/ChatGPT/comments/1dsj...,1,2024-07-01
142,117656,Reddit,Told ChatGPT to stop replying to me and it thr...,https://www.reddit.com/r/ChatGPT/comments/1dsi...,1,2024-07-01


In [44]:
print(df.iloc[143].url)


https://www.theregister.com/2024/07/01/database_ai_training/


In [89]:

# takes 5 minutes without multiprocessing
# get driver takes 50 seconds, so call it 1 + 4
# 2 drivers should take 2 + 2, 3: 3 + 4/3
# could save a minute by using 2 webdrivers

# Fetch HTML files from sources
# empty download directory
DOWNLOAD_DIR = webscrape.DOWNLOAD_DIR

# Iterate through the files in the directory and delete them
for filename in os.listdir(DOWNLOAD_DIR):
    file_path = os.path.join(DOWNLOAD_DIR, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print(f'Failed to delete {file_path}. Reason: {e}')

print(f"All files in {DOWNLOAD_DIR} have been deleted.")

# save each file specified from sources
print("Saving HTML files")


All files in htmldata have been deleted.
Saving HTML files


In [90]:
# Create a queue for multiprocessing and populate it 
queue = multiprocessing.Queue()
for row in df.itertuples():
    queue.put((row.url, row.title))

In [73]:
# webscrape.get_url('https://www.theregister.com/2024/07/01/database_ai_training/')
# 

'CTOs__frustration_with_GenAI__Tool_changes_that_break_stuff___The_Register_20240701_082517.html'

In [91]:
# Function to take the queue and pop entries off and process until none are left
# lets you create an array of functions with different args
def process_queue_factory(q):
    def process_queue():
        # launch browser via selenium driver
        driver = webscrape.get_driver()    
        saved_pages = []
        while not q.empty():
            url, title = q.get()
            webscrape.log(f'Processing {url}')
            savefile = webscrape.get_url(url, driver)
            saved_pages.append((url, title, savefile))
        # Close the browser
        webscrape.log("Quit webdriver")
        driver.quit()
        return saved_pages
    return process_queue

callable = process_queue_factory(queue)
# saved_pages = callable()

def launch_drivers(num_drivers):
    with ThreadPoolExecutor(max_workers=num_drivers) as executor:
        # Create a list of future objects
        futures = [executor.submit(callable) for _ in range(num_drivers)]
        
        # Collect the results (web drivers) as they complete
        retarray = [future.result() for future in as_completed(futures)]
        
    return retarray

results = launch_drivers(4)


In [92]:
# flatten results
saved_pages = []
for r in results:
    saved_pages.extend(r)
saved_pages


[('https://www.bloomberg.com/news/articles/2024-07-01/turkey-turns-to-ai-to-crack-down-on-rampant-tax-evasion',
  'Turkey Turns to AI to Crack Down on Rampant Tax Evasion',
  'htmldata/Turkey_Turns_to_AI_to_Crack_Down_on_Rampant_Tax_Evasion_-_Bloomberg_20240701_091042.html'),
 ('https://prospect.org/environment/2024-07-01-unknown-toll-of-ai-takeover/',
  'The Unknown Toll of the AI Takeover',
  'htmldata/The_Unknown_Toll_of_the_AI_Takeover_-_The_American_Prospect_20240701_091053.html'),
 ('https://www.techradar.com/pro/ai-pcs-is-it-all-hype-should-you-switch-and-if-so-when',
  'AI PCs. Is it all hype, should you switch, and if so, when?',
  'htmldata/AI_PCs__Is_it_all_hype__should_you_switch__and_if_so__when____TechRadar_20240701_091104.html'),
 ('https://www.zdnet.com/article/generative-ai-is-new-attack-vector-endangering-enterprises-says-crowdstrike-cto/',
  'Generative AI is new attack vector endangering enterprises, says CrowdStrike CTO',
  'htmldata/Generative_AI_is_new_attack_vec

In [96]:
len(saved_pages)

144

In [97]:
files = {f for f in os.listdir(DOWNLOAD_DIR) if f.endswith('html')}
files
len(files)


133

In [107]:
xf = []
for f in saved_pages:
    if f[2].startswith('htmldata/'):
        xf.append(f[2][9:])
    else:
        xf.append(f[2])
    

In [ ]:
# set(xf).intersection(set(files))


In [ ]:
# files

In [ ]:
# summarize pages
SYSTEM_PROMPT = """You are a summarization assistant. 
You will summarize the main content of provided text from HTML files in 3 bullet points or less.
You will output Markdown format.
You will ignore any content that appears to be navigation menus, footers, sidebars, or other boilerplate content.
You will provide the bullet points only, without any introduction such as 'here are' or any conclusion, or comment.
"""

local_llm = ChatOllama(model='gemma2', temperature=0)

all_responses = ""

for url, title, filename in saved_pages:
    # Read the HTML file
    try:
        with open(filename, 'r', encoding='utf-8') as file:
            html_content = file.read()
    except:
        print(f"Skipping {filename}")
        continue

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')

    # Filter out script and style elements
    for script_or_style in soup(['script', 'style']):
        script_or_style.extract()

    # Get text and strip leading/trailing whitespace
    visible_text = soup.get_text(separator=' ', strip=True)

    userprompt = """Summarize the main points of the following text in at most 3 bullet points:
{text}
"""
    prompt_template = ChatPromptTemplate.from_messages(
        [("system", SYSTEM_PROMPT),
         ("user", userprompt)]
    )
    parser = StrOutputParser()

    chain = prompt_template | local_llm | parser

    response = chain.invoke({'text': visible_text})
    
    output = f"[{title}]({url})\n\n"
    output += response
    output += "-" * 10
    output += "\n\n"
    display(Markdown(output))

    all_responses += output



[Turkey Turns to AI to Crack Down on Rampant Tax Evasion](https://www.bloomberg.com/news/articles/2024-07-01/turkey-turns-to-ai-to-crack-down-on-rampant-tax-evasion)

- Turkey is using AI to combat tax evasion, aiming to identify wrongdoing in companies that report losses or "symbolic" profits.
- The country is lagging behind other OECD members in tax collection as a percentage of its economic output.
-  The government is pushing for new taxes to bolster finances after last year's earthquakes. 
----------



[The Unknown Toll of the AI Takeover](https://prospect.org/environment/2024-07-01-unknown-toll-of-ai-takeover/)

This article delves into the hidden costs of artificial intelligence (AI), particularly focusing on its energy consumption and environmental impact. 

Here are the key takeaways:

* **AI's Energy Hunger:** AI, especially large language models, require massive amounts of energy to operate. This demand is rapidly increasing, straining power grids and contributing to climate change.
* **Data Centers as Energy Guzzlers:** Data centers, where AI is processed, are major energy consumers. They often receive special deals from utilities, shifting costs onto ratepayers and hindering the transition to renewable energy.
* **Lobbying and Regulatory Gaps:** Tech companies heavily lobby against regulations that would address AI's environmental impact. This creates a regulatory gap, allowing companies to prioritize profits over sustainability.
* **Unequal Burden:** The environmental costs of AI are disproportionately borne by developing countries, where data centers are increasingly built, exploiting lower energy and water prices.
* **Hidden Costs:** Consumers often don't realize the link between their online activity and the energy consumption of AI. This lack of transparency allows companies to avoid accountability.
* **Call for Action:** The article urges for greater transparency, regulation, and public awareness about AI's environmental impact. It calls for a shift towards sustainable AI development and a pricing mechanism, like a carbon tax, to reflect the true cost of energy consumption.

**Overall, the article paints a concerning picture of AI's environmental impact, highlighting the need for urgent action to mitigate its negative consequences.**


Let me know if you have any other questions.----------



[AI PCs. Is it all hype, should you switch, and if so, when?](https://www.techradar.com/pro/ai-pcs-is-it-all-hype-should-you-switch-and-if-so-when)

- AI PCs feature a dedicated Neural Processing Unit (NPU) that handles AI applications, offering low latency and personalization.
- AI PCs combine CPU, GPU, and NPU for flexible computing, enabling diverse applications and enhancing performance.
-  The adoption of AI PCs is growing, driven by benefits like improved security, privacy, and offline functionality, but requires careful consideration of memory, storage, and staff training needs. 
----------



[Generative AI is new attack vector endangering enterprises, says CrowdStrike CTO](https://www.zdnet.com/article/generative-ai-is-new-attack-vector-endangering-enterprises-says-crowdstrike-cto/)

- Generative AI (GenAI) is a new attack vector for enterprises, as it can be used to obtain sensitive data without malware.
- GenAI programs are vulnerable to prompt injection attacks, which can reveal sensitive information that has been used to train the model.
- Enterprises need to implement robust security controls, such as validating user prompts and responses, to mitigate the risks of GenAI. 


----------



[Using AI to Fight Trafficking Is Dangerous](https://www.hrw.org/news/2024/07/01/using-ai-fight-trafficking-dangerous)

- The US State Department encourages technology companies to use AI to detect human trafficking patterns, but this approach is dangerous.
- AI language models are likely to be built on discriminatory stereotypes that have plagued anti-trafficking efforts, leading to the disproportionate surveillance of marginalized groups.
-  Digital tools and policies that take a broad approach to collecting evidence of alleged exploitation online are dangerous and counterproductive, as they can lead to overreporting and overwhelm law enforcement. 
----------



...

[House lawmakers push measure to limit impact of deceptive AI on elections](https://news.google.com/articles/CBMigAFodHRwczovL3d3dy5uZXh0Z292LmNvbS9hcnRpZmljaWFsLWludGVsbGlnZW5jZS8yMDI0LzA3L2hvdXNlLWxhd21ha2Vycy1wdXNoLW1lYXN1cmUtbGltaXQtaW1wYWN0LWRlY2VwdGl2ZS1haS1lbGVjdGlvbnMvMzk3NzU5L9IBAA)

- A bill introduced by Rep. Shontel Brown aims to limit the use of AI in elections by prohibiting its use to interfere with voting rights and requiring disclaimers on AI-generated election content.
- The legislation would penalize individuals or organizations found to be violating these provisions with fines or imprisonment.
- Lawmakers are concerned about the potential for AI to spread misinformation and influence elections, citing examples like AI-generated robocalls and the need for transparency in political advertising. 
----------



[The Chinese Army trains an AI to reproduce the decisions of military leaders](https://news.google.com/articles/CBMiSWh0dHBzOi8vbWV0YS1kZWZlbnNlLmZyL2VuLzIwMjQvMDYvMjUvQ2hpbmVzZS1hcm15LXdhcmdhbWUtQUktc2ltdWxhdGlvbi_SAQA)

- The Chinese Army is using AI to simulate the decisions of military leaders in wargames to improve training realism.
- This AI is designed to replicate the behavior and biases of both Chinese and foreign military leaders, overcoming the limitations of human trainers.
- The goal is to create more realistic wargames that better prepare PLA officers for potential conflicts. 
----------



[Apple Intelligence may get Google Gemini support in the next few months](https://news.google.com/articles/CBMiTmh0dHBzOi8vd3d3LnhkYS1kZXZlbG9wZXJzLmNvbS9hcHBsZS1pbnRlbGxpZ2VuY2UtZ29vZ2xlLWdlbWluaS1zdXBwb3J0LXJ1bW9yL9IBAA)

- Apple Intelligence may integrate Google Gemini as a third-party AI option alongside ChatGPT and Siri.
- This strategy allows users to choose their preferred AI model within Apple Intelligence.
- Apple's approach aims to unite AI models rather than compete, potentially making Apple Intelligence the preferred platform for AI tools. 
----------



[Promising directions of machine learning for partial differential equations](https://news.google.com/articles/CBMiMmh0dHBzOi8vd3d3Lm5hdHVyZS5jb20vYXJ0aWNsZXMvczQzNTg4LTAyNC0wMDY0My0y0gEA)

This text appears to be a list of scientific publications related to machine learning and its application to solving partial differential equations (PDEs). 

Here's a breakdown of what the text likely represents:

* **Citations:** Each entry is a citation for a scientific article. It includes author names, title, journal, year of publication, and sometimes links to online resources like MathSciNet or arXiv.
* **Focus:** The articles primarily focus on using machine learning techniques to:
    * **Solve PDEs:** This involves using algorithms to find solutions to mathematical equations that describe physical phenomena like fluid flow, heat transfer, and wave propagation.
    * **Learn PDE operators:** This means training machine learning models to represent the mathematical operators that appear in PDEs.
    * **Accelerate PDE simulations:** Using machine learning to speed up the process of solving PDEs, which can be computationally expensive.
* **Techniques:** The text mentions several machine learning techniques used in this context, including:
    * **Neural operators:** A type of neural network specifically designed to learn PDE operators.
    * **Fourier neural operators:** A variant of neural operators that uses Fourier transforms to represent the PDE.
    * **Transformer networks:** A type of neural network architecture known for its ability to handle sequential data, which can be applied to time-dependent PDEs.
    * **Reservoir computing:** A technique for predicting the future behavior of complex systems based on past data.
* **Applications:** The text also highlights applications of these techniques in various fields, such as:
    * **Computational fluid dynamics:** Simulating the flow of fluids.
    * **Weather forecasting:** Predicting future weather patterns.
    * **Plasma physics:** Modeling the behavior of plasmas.

Overall, this text provides a glimpse into the exciting research happening at the intersection of machine learning and PDEs, with the potential to revolutionize scientific computing and our understanding of complex physical systems.


----------



[China's Killer Robots Are Coming](https://news.google.com/articles/CBMiRWh0dHBzOi8vd3d3Lm5ld3N3ZWVrLmNvbS9jaGluYS1raWxsZXItcm9ib3RzLXVuaXRyZWUtcm9ib3RpY3MtMTkxNzU2OdIBAA)

- China is developing autonomous "killer robots" at a rapid pace, potentially deploying them within two years.
- China and Russia are collaborating on AI-powered autonomous weaponry, showcasing such systems in recent military drills.
- Experts warn that the development of autonomous weapons poses a significant threat to humanity, potentially removing ethical considerations from warfare. 
----------

